In [2]:
import collections
import os
import shutil
import numpy as np

In [3]:
chrom_list = []
for i in range(1, 23):
    chrom_list.append(str(i))
chrom_list.append('X')
chrom_list.append('Y')

In [4]:
# with open('/autofs/bal20/ykzhou/HG00733_ONT_PanGenome/snp_8x_hg38/snp_pass.vcf', 'r') as file:
home = '/autofs/bal20/ykzhou/HG00733/'
# aln_file = home + 'alns/HG00733_1_Guppy_4.2.2_prom_8X.bam'

# ref_file = '/autofs/bal20/ykzhou/ref/hg38.no_alt.fa'

snp_home = home + 'snp_8x_012/'
phase_home = snp_home + 'phase/'
pass_home = phase_home + 'pass/'
thres_home = phase_home + 'thres/'
all_home = phase_home + 'all/'
pileup_home = phase_home + 'pileup/'

os.system('mkdir ' + phase_home + ' ' + pass_home + ' ' + thres_home + ' ' + all_home + ' ' + pileup_home)

0

In [5]:
with open(snp_home + 'merge_output.vcf', 'r') as file:
    vcf_raw = file.readlines()
vcf_raw = [s.strip() for s in vcf_raw]
vcf_raw = [vcf_raw[i].split() for i in range(len(vcf_raw))]
snp_passcall = [vcf_raw[i] for i in range(len(vcf_raw)) if vcf_raw[i][0][:3] == 'chr']

for snp in range(len(snp_passcall)):
    snp_passcall[snp].append(float(snp_passcall[snp][5]))
    
def score(snp):
    return snp[10]

snp_passcall.sort(key = score)

In [ ]:
splitted: chr.vcf
sorted: chr.vcf
phased: phased_chr.vcf
bam: chr.bam
sam: chr.sam

In [6]:
# thres

precision = 0.9
cutline = round(len(snp_passcall) * (1 - precision))

snp_header = [vcf_raw[i] for i in range(len(vcf_raw)) if vcf_raw[i][0][:3] != 'chr']

fd = thres_home
for ch in range(24):
    file = open(fd + 'chr' + chrom_list[ch] + '.vcf', 'w')
    for line in snp_header:
        if line[0] == '#CHROM':
            st = '\t'.join(line) + '\n'
        else:
            st = ' '.join(line) + '\n'
        file.write(st)
    chrom_snp = [snp[:-1] for snp in snp_passcall[cutline:] if snp[0] == 'chr' + chrom_list[ch]]
    for line in chrom_snp:
        st = '\t'.join(line) + '\n'
        file.write(st)
    file.close()
    
for ch in range(24):
    vcf = fd + 'chr' + chrom_list[ch] + '.vcf'
    sort_vcf = fd + 'sorted_chr' + chrom_list[ch] + '.vcf'
    os.system('bcftools sort ' + vcf + ' > ' + sort_vcf)

In [7]:
# pass

fd = pass_home
for ch in range(24):
    file = open(fd + 'chr' + chrom_list[ch] + '.vcf', 'w')
    for line in snp_header:
        if line[0] == '#CHROM':
            st = '\t'.join(line) + '\n'
        else:
            st = ' '.join(line) + '\n'
        file.write(st)
    chrom_snp = [snp[:-1] for snp in snp_passcall if snp[0] == 'chr' + chrom_list[ch] and snp[6] == 'PASS']
    for line in chrom_snp:
        st = '\t'.join(line) + '\n'
        file.write(st)
    file.close()
    
for ch in range(24):
    vcf = fd + 'chr' + chrom_list[ch] + '.vcf'
    sort_vcf = fd + 'sorted_chr' + chrom_list[ch] + '.vcf'
    os.system('bcftools sort ' + vcf + ' > ' + sort_vcf)

In [8]:
# pileup

fd = pileup_home
for ch in range(24):
    file = open(fd + 'chr' + chrom_list[ch] + '.vcf', 'w')
    for line in snp_header:
        if line[0] == '#CHROM':
            st = '\t'.join(line) + '\n'
        else:
            st = ' '.join(line) + '\n'
        file.write(st)
    chrom_snp = [snp[:-1] for snp in snp_passcall if snp[0] == 'chr' + chrom_list[ch] and snp[7] == 'P']
    for line in chrom_snp:
        st = '\t'.join(line) + '\n'
        file.write(st)
    file.close()
    
for ch in range(24):
    vcf = fd + 'chr' + chrom_list[ch] + '.vcf'
    sort_vcf = fd + 'sorted_chr' + chrom_list[ch] + '.vcf'
    os.system('bcftools sort ' + vcf + ' > ' + sort_vcf)

In [9]:
# all

fd = all_home
for ch in range(24):
    file = open(fd + 'chr' + chrom_list[ch] + '.vcf', 'w')
    for line in snp_header:
        if line[0] == '#CHROM':
            st = '\t'.join(line) + '\n'
        else:
            st = ' '.join(line) + '\n'
        file.write(st)
    chrom_snp = [snp[:-1] for snp in snp_passcall if snp[0] == 'chr' + chrom_list[ch]]
    for line in chrom_snp:
        st = '\t'.join(line) + '\n'
        file.write(st)
    file.close()
    
for ch in range(24):
    vcf = fd + 'chr' + chrom_list[ch] + '.vcf'
    sort_vcf = fd + 'sorted_chr' + chrom_list[ch] + '.vcf'
    os.system('bcftools sort ' + vcf + ' > ' + sort_vcf)

In [52]:
os.system('parallel -j40 "whatshap phase -o ' + pass_home + 'phased_{1}.vcf.gz -r ' + ref_file + 
          ' --chromosome {1} --distrust-genotypes --ignore-read-groups --output-read-list ' + 
          pass_home + 'read_{1}.txt ' + pass_home + 'sorted_{1}.vcf ' + aln_file + '" ::: ${CHR[@]}')

0

In [96]:
9**6, 5**8

(531441, 390625)

CHR=(chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY)
HOME="/autofs/bal20/ykzhou/HG00733/"
SNP_HOME=${HOME}"snp_8x_012/"
PHASE_HOME=${SNP_HOME}"phase/"
PASS_HOME=${PHASE_HOME}"pass/"
THRES_HOME=${PHASE_HOME}"thres/"
ALL_HOME=${PHASE_HOME}"all/"
PILEUP_HOME=${PHASE_HOME}"pileup/"
cd ${PASS_HOME}
for i in ${CHR}
do
  samtools view -h -@40 ${i}".bam" > ${i}".sam"
done
cd ${ALL_HOME}
for i in ${CHR}
do
  samtools view -h -@40 ${i}".bam" > ${i}".sam"
done
cd ${PILEUP_HOME}
for i in ${CHR}
do
  samtools view -h -@40 ${i}".bam" > ${i}".sam"
done
cd ${THRES_HOME}
for i in ${CHR}
do
  samtools view -h -@40 ${i}".bam" > ${i}".sam"
done

In [15]:
fil = [snp[7] for snp in snp_passcall][:cutline]
tru = [snp[7] for snp in snp_passcall][cutline + 1:]

In [35]:
snp_passcall[cutline][-1]

6.58

In [20]:
len([s for s in tru if s == 'F']) / len(tru)

0.08826138459188043

In [24]:
lowq = [snp[-1] for snp in snp_passcall if snp[6] == 'LowQual']
len(lowq)

22586

In [ ]:
# pass_snp, thres_snp, all_snp